In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import cplex
import optlang
import math
import json
import sys
import cProfile
from optlang.symbolics import Zero, add
import cobra.util.solver as sutil
from cobra.flux_analysis.parsimonious import pfba
from IPython.core.display import HTML
import cobrakbase
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import FlexibleBiomassPkg, ElementUptakePkg, SimpleThermoPkg, ReactionUsePkg, BilevelPkg, CommKineticPkg,KBaseMediaPkg


cobrakbase 0.2.7


In [2]:
kbase_api = cobrakbase.KBaseAPI()

model = kbase_api.get_from_ws("E_iML1515.kb",40576)
model.solver = 'optlang-cplex'

#Applying uptake limit to total carbon uptake by model
eup = ElementUptakePkg(model)
eup.build_package({"C":60})
#Applying constraints based on KBase media
media = kbase_api.get_from_ws("Btheta_Ecoli_minimal_media",40576)
kmp = KBaseMediaPkg(model)
kmp.build_package(media)
#Applying flexible biomass constraints
fbp = FlexibleBiomassPkg(model)
fbp.build_package({"bio_rxn_id":"bio1"})
#Setting the objective to biomass
biomass_objective = model.problem.Objective(
    1 * model.reactions.bio1.flux_expression,
    direction='max')
model.objective = biomass_objective
#Printing LP file
with open('FlexibleBiomass.lp', 'w') as out:
    out.write(str(model.solver))
#Solving
sol=model.optimize()
model.summary()

/Users/chenry/opt/anaconda3/lib/python3.7/site-packages/cobra/core/metabolite.py:86 UserWarning: invalid character '*' found in formula '*'
/Users/chenry/opt/anaconda3/lib/python3.7/site-packages/cobra/core/metabolite.py:86 UserWarning: invalid character '*' found in formula 'C12H20O10*2'


rna:cpd00002_c0:504.157206:-88.22751104999857:-0.17499999999999716
rna:cpd00052_c0:480.13250600000003:-152.32904166104657:-0.133508
rna:cpd00038_c0:520.156606:-264.2126469852226:-0.215096
rna:cpd00062_c0:481.117266:-333.5435694848866:-0.144104
rna:cpd00012_c0:174.951264:-216.72721090197507:0.6677079999999971
rna:cpd00001_c0:18.01528:-204.69826432373512:0.6677079999999971
dna:cpd00115_c0:488.157806:-12.773137151795998:-0.026166
dna:cpd00356_c0:464.133106:-25.312621276597998:-0.027017
dna:cpd00241_c0:504.157206:-38.9334365111:-0.027017
dna:cpd00357_c0:479.144446:-51.470730085136:-0.026166
dna:cpd00012_c0:174.951264:-32.861863938512:0.10636599999999999
dna:cpd00001_c0:18.01528:-30.945650666032:0.10636599999999999
protein:cpd00035_c0:89.09318:-45.76618654102:-0.513689
protein:cpd00051_c0:175.2089:-97.59157748982:-0.295792
protein:cpd00132_c0:132.11792:-129.43926269541998:-0.241055
protein:cpd00041_c0:132.09474:-161.28136024612:-0.241055
protein:cpd00084_c0:121.15818:-172.37702637051999:-0.

In [3]:
for reaction in fbp.new_reactions:
    print(reaction+":"+str(fbp.new_reactions[reaction].forward_variable.primal-fbp.new_reactions[reaction].reverse_variable.primal))

FLEX_cpd00201_c0:-3.2949651919770296e-05
FLEX_cpd24604_c0:-3.841663452529272e-06
FLEX_cpd03445_c0:-3.2949651919770296e-05
FLEX_cpd24682_c0:-3.8416634525292714e-05
FLEX_cpd00035_c0:0.07590077912562727
FLEX_cpd00017_c0:-3.2949651919770296e-05
FLEX_cpd00051_c0:-0.04370512753655917
FLEX_cpd00132_c0:0.0356173916749786
FLEX_cpd00041_c0:0.0356173916749786
FLEX_cpd00104_c0:-3.940167643619766e-07
FLEX_cpd00063_c0:0.0007690714719390332
FLEX_cpd00099_c0:0.0007690714719390332
FLEX_cpd00010_c0:-8.510762110218694e-05
FLEX_cpd00149_c0:3.693907165893531e-06
FLEX_cpd00052_c0:0.01972664631616454
FLEX_cpd00058_c0:0.00010475920722474053
FLEX_cpd00084_c0:-0.013531520730101183
FLEX_cpd00115_c0:-0.003866190996110805
FLEX_cpd00356_c0:0.003991931596037821
FLEX_cpd00241_c0:-0.0013306438653459401
FLEX_cpd00015_c0:-3.2949651919770296e-05
FLEX_cpd10515_c0:0.0009921834647590023
FLEX_cpd10516_c0:0.0011536810860518676
FLEX_cpd00053_c0:0.03888354439106166
FLEX_cpd00023_c0:0.03888354439106166
FLEX_cpd00033_c0:-0.004288